### **문제 1) Tokenizer 생성하기**

In [ ]:
import pandas as pd
from math import log
import re


class Tokenizer():
    def __init__(self):
        self.word_dict = {'oov': 0}
        self.fit_checker = False

        self.preprocess = []

    def preprocessing(self, sequences):
        '''
        문제 1-1.
        '''
        self.preprocess = [re.sub(f'[^a-zA-Z\s]', '', i.lower()).split()
                           for i in sequences]

        return self.preprocess

    def fit(self, sequences):
        self.fit_checker = False
        '''
        문제 1-2.
        '''
        sequences = self.preprocessing(sequences)
        idx = 1
        for i in sequences:
            for j in i:
                if j not in self.word_dict:
                    self.word_dict[j] = idx
                    idx += 1

        self.fit_checker = True

    def transform(self, sequences):
        result = []
        tokens = self.preprocessing(sequences)
        if self.fit_checker:
            '''
            문제 1-3.
            '''
            for i in tokens:
                ls = []
                for j in i:
                    if j in self.word_dict:
                        ls.append(self.word_dict[j])
                    else:
                        ls.append(self.word_dict['oov'])
                result.append(ls)

            return result
        else:
            raise Exception("Tokenizer instance is not fitted yet.")

    def fit_transform(self, sequences):
        self.fit(sequences)
        result = self.transform(sequences)
        return result

**1-1. `preprocessing()`**

텍스트 전처리를 하는 함수입니다.

- input: 여러 영어 문장이 포함된 list 입니다. ex) ['I go to school.', 'I LIKE pizza!']
- output: 각 문장을 토큰화한 결과로, nested list 형태입니다. ex) [['i', 'go', 'to', 'school'], ['i', 'like', 'pizza']]
- 조건 1: 입력된 문장에 대해서 소문자로의 변환과 특수문자 제거를 수행합니다.
- 조건 2: 토큰화는 white space 단위로 수행합니다.

In [ ]:
token = Tokenizer()
input_list = ['I go to school.', 'I LIKE pizza!']
token_list = token.preprocessing(input_list)
token_list

**1-2. `fit()`**

어휘 사전을 구축하는 함수입니다.

- input: 여러 영어 문장이 포함된 list 입니다. ex) ['I go to school.', 'I LIKE pizza!']
- 조건 1: 위에서 만든 `preprocessing` 함수를 이용하여 각 문장에 대해 토큰화를 수행합니다.
- 조건 2: 각각의 토큰을 정수 인덱싱 하기 위한 어휘 사전(`self.word_dict`)을 생성합니다.
    - 주어진 코드에 있는 `self.word_dict`를 활용합니다.

In [ ]:
token.word_dict

In [ ]:
token.fit(input_list)
token.word_dict

**1-3. `transform()`**

어휘 사전을 활용하여 입력 문장을 정수 인덱싱하는 함수입니다.

- input: 여러 영어 문장이 포함된 list입니다. ex) ['I go to school.', 'I LIKE pizza!']
- output: 각 문장의 정수 인덱싱으로, nested list 형태입니다. ex) [[1, 2, 3, 4], [1, 5, 6]]
- 조건 1: 어휘 사전(`self.word_dict`)에 없는 단어는 'oov'의 index로 변환합니다.

In [ ]:
input_list2 = ['I go to school.', 'I LIKE pizza!', 'one TO three']
output = token.transform(input_list2)
print(f'word_dict : {token.word_dict}')
print(f'output : {output}')

### **문제 2) TfidfVectorizer 생성하기**

In [ ]:
class TfidfVectorizer:
    def __init__(self, tokenizer):
        self.tokenizer = tokenizer
        self.fit_checker = False
        self.tfidf_matrix = []
        self.idf = []
        self.tf = []

    def fit(self, sequences):
        tokenized = self.tokenizer.fit_transform(sequences)
        n = len(tokenized)
        words = list(self.tokenizer.word_dict.keys())[1:]
        word_words = self.tokenizer.preprocess
        '''
        문제 2-1.
        '''
        for word in words:
            s = 0
            for i in word_words:
                if word in i:
                    s += 1

            self.idf.append(log(n/(s+1)))

        self.fit_checker = True
        
    def transform(self, sequences):
        if self.fit_checker:
            tokenized = self.tokenizer.transform(sequences)
            n = len(tokenized)
            words = list(self.tokenizer.word_dict.keys())[1:]
            word_words = self.tokenizer.preprocess
            '''
            문제 2-2.
            '''
            for i in word_words:
                self.tf.append([])
                for word in words:
                    self.tf[-1].append(i.count(word))

            for i in range(n):
                cur = []
                for j in range(len(words)):
                    cur.append(self.tf[i][j] * self.idf[j])
                self.tfidf_matrix.append(cur)
            return self.tfidf_matrix

        else:
            raise Exception("TfidfVectorizer instance is not fitted yet.")

    def fit_transform(self, sequences):
        self.fit(sequences)
        return self.transform(sequences)

**2-1. `fit()`**

입력 문장들을 이용해 IDF 행렬을 만드는 함수입니다.

- input: 여러 영어 문장이 포함된 list 입니다. ex) ['I go to school.', 'I LIKE pizza!']
- 조건 1: IDF 행렬은 list 형태입니다.
    - ex) [토큰1에 대한 IDF 값, 토큰2에 대한 IDF 값, .... ]
- 조건 2: IDF 값은 아래 식을 이용해 구합니다.
    
    $$
    idf(d,t)=log_e(\frac{n}{1+df(d,t)})
    $$
    
    - $df(d,t)$ : 단어 t가 포함된 문장 d의 개수
    - $n$ : 입력된 전체 문장 개수
- 조건 3: 입력된 문장의 토큰화에는 문제 1에서 만든 Tokenizer를 사용합니다.

In [ ]:
tfidf_vec = TfidfVectorizer(token)
sequences = ['I go to school.', 'I LIKE pizza!']

In [ ]:
tfidf_vec.fit(sequences)
print(f'IDF : {tfidf_vec.idf}')

**2-2. `transform()`**

입력 문장들을 이용해 TF-IDF 행렬을 만드는 함수입니다.

- input: 여러 영어 문장이 포함된 list입니다. ex) ['I go to school.', 'I LIKE pizza!']
- output : nested list 형태입니다.
    
    ex) [[tf-idf(1, 1), tf-idf(1, 2), tf-idf(1, 3)], [tf-idf(2, 1), tf-idf(2, 2), tf-idf(2, 3)]]
    
    |  | 토큰1 | 토큰2 | 토큰3 |
    | --- | --- | --- | --- |
    | 문장1 | tf-idf(1,1) | tf-idf(1,2) | tf-idf(1,3) |
    | 문장2 | tf-idf(2,1) | tf-idf(2,2) | tf-idf(2,3) |
- 조건1 : 입력 문장을 이용해 TF 행렬을 만드세요.
    - $tf(d, t)$ : 문장 d에 단어 t가 나타난 횟수
- 조건2 : 문제 2-1( `fit()`)에서 만든 IDF 행렬과 아래 식을 이용해 TF-IDF 행렬을 만드세요
    
    $$
    tf-idf(d,t) = tf(d,t) \times idf(d,t)
    $$

In [ ]:
tfidf = tfidf_vec.transform(sequences)
print(f'TF : {tfidf_vec.tf}')

In [ ]:
print(f'TF-IDF : {tfidf}')

## ★ 최종 결과

In [ ]:
columns = list(tfidf_vec.tokenizer.word_dict.keys())[1:]
pd_tfidf_matrix = pd.DataFrame(tfidf_vec.tfidf_matrix, columns=columns)
print(pd_tfidf_matrix)